In [1]:
from PIL import Image

from keras.models import load_model
import numpy as np
from numpy import asarray
from numpy import expand_dims

import pickle
import cv2

In [2]:
#HaarCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))


In [3]:
import numpy as np
import tensorflow as tf
import keras

In [4]:
pip install keras-facenet


[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from keras_facenet import FaceNet
embedder = FaceNet()

In [6]:
myfile = open("data.pkl", "rb")
database = pickle.load(myfile)
myfile.close()

In [8]:
cap = cv2.VideoCapture(0)

while(1):
    _, gbr1 = cap.read()
    
    wajah = HaarCascade.detectMultiScale(gbr1,1.1,4)
    
    if len(wajah)>0:
        x1, y1, width, height = wajah[0]        
    else:
        x1, y1, width, height = 1, 1, 10, 10
    
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)                  # konversi dari OpenCV ke PIL
    gbr_array = asarray(gbr)
    
    face = gbr_array[y1:y2, x1:x2]                        
    
    face = Image.fromarray(face)                       
    face = face.resize((160,160))
    face = asarray(face)
    
    
    face = expand_dims(face, axis=0)
    signature = embedder.embeddings(face)
    
    min_dist=100
    identity=' '
    for key, value in database.items() :
        dist = np.linalg.norm(value-signature)
        if dist < min_dist:
            min_dist = dist
            identity = key
            
    cv2.putText(gbr1,identity, (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.rectangle(gbr1,(x1,y1),(x2,y2), (0,255,0), 2)
        
    cv2.imshow('res',gbr1)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 351ms/step


In [8]:
import numpy as np

# Assuming you have 'database' with known identities and corresponding face embeddings
# Assuming 'test_data' contains the test images and corresponding face embeddings

true_positive = 0
total_faces = len(database)

for test_identity, test_embedding in database.items():
    min_dist = float('inf')
    recognized_identity = None
    
    for key, database_embedding in database.items():
        dist = np.linalg.norm(test_embedding - database_embedding)
        if dist < min_dist:
            min_dist = dist
            recognized_identity = key
    
    if recognized_identity == test_identity:
        true_positive += 1

accuracy = (true_positive / total_faces) * 100
print(f"FaceNet model accuracy: {accuracy:.2f}%")


FaceNet model accuracy: 100.00%
